In [1]:
from pyPept.sequence import Sequence
from pyPept.sequence import correct_pdb_atoms
from pyPept.molecule import Molecule
from pyPept.converter import Converter
from pyPept.conformer import Conformer
from pyPept.conformer import SecStructPredictor

from rdkit import Chem
from rdkit.Chem import Draw

#NGLview

import nglview as ng

import os 
import time 

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:

aaseq = input("Enter Amino Acid Sequence")
biln = "-".join(aaseq)

print(biln)

seq = Sequence(biln)
# Correct atom names in the sequence object
seq = correct_pdb_atoms(seq)

# Loop wit the included monomers
mm_list = seq.s_monomers
for i, monomer in enumerate(mm_list):
    mon = monomer['m_romol']

# Generate the RDKit object
mol = Molecule(seq, depiction='rdkit')
romol = mol.get_molecule(fmt='ROMol')
print("The SMILES of the peptide is: {}".format(Chem.MolToSmiles(romol)))
Draw.MolToFile(romol, 'peptidemol.png', size=(1200, 1200))

# Create the peptide conformer with corrected atom names and secondary structure
# Obtain peptide main chain to predict the secondary structure
fasta = Conformer.get_peptide(biln)
secstruct = SecStructPredictor.predict_active_ss(fasta)
# Generate the conformer
romol = Conformer.generate_conformer(romol, secstruct, generate_pdb=True)


Enter Amino Acid Sequence ARLLRRAGG


Failed to patch pandas - unable to change molecule rendering


A-R-L-L-R-R-A-G-G


C:\Users\Daniel\anaconda3\envs\pypept\lib\site-packages\pyPept\sequence.py:536: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_group[group][idx] = updated_change
Failed to patch pandas - unable to change molecule rendering
Failed to patch 

The SMILES of the peptide is: CC(C)C[C@H](NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](C)N)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](C)C(=O)NCC(=O)NCC(=O)O
Predicted Secondary Structure: -HHHHHHH- for main chain: ARLLRRAGG


In [3]:

view = ng.show_structure_file("structure.pdb")
view.add_representation('cartoon', selection='protein')
view

NGLWidget()

In [4]:
def generate_images(v=view):
    v.clear()
    v.add_cartoon(color='red')
    im0 = v.render_image()
    v.clear()
    v.add_cartoon(color='blue')
    im1 = v.render_image()
    for im in [im0, im1]:
        while not im.value:
            time.sleep(0.1)
    for n, im in zip('ab', [im0, im1]):
        with open(f'figure_{n}.png', 'wb') as fh:
            fh.write(im.value)

import threading
thread = threading.Thread(
    target=generate_images,
)
thread.daemon = True
thread.start()